In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [2]:
df=pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')
df

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


In [3]:
macros_data=[]
bmr=[]
tdee=[]
bmi=[]
for height, weight, gender, age, phy_activity,obesity in tqdm(zip(df['Height'], df['Weight'],df['Gender'], df['Age'],df['FAF'],df['NObeyesdad'])):
    url = "https://www.calculator.net/macro-calculator.html"
    height_cm=height*100
    if gender=="Male":
        gender_par="m"
        bmr_1= 88.362 + (13.397 * weight) + (4.799 * height_cm) - (5.677 * age)
    else:
        gender_par="f"
        bmr_1= 447.593 + (9.247 * weight ) + (3.098 * height_cm) -(4.330 * age)

    bmr.append(bmr_1)

    bmi_1=weight/(height**2)
    bmi.append(bmi_1)
    
    if phy_activity== 0:
        td= 1.2
    elif phy_activity==1:
        td= 1.375
    elif phy_activity==2:
        td= 1.465
    elif phy_activity==3:
        td==1.55
    elif phy_activity==4:
        td= 1.725
    else:
        td== 1

    td_1=td*bmr_1
    tdee.append(td_1)

    if obesity=='Insufficient_Weight':
        cg='g1'
    elif obesity=='Overweight_Level_I' or obesity=='Overweight_Level_II':
        cg='l'
    elif obesity=='Obesity_Level_I':
        cg='l1'
    elif obesity=='Obesity_Level_II' or obesity=='Obesity_Level_III':
        cg='l2'
    else:
        cg='m'
        

    params = {
        "ctype": "metric",
        "cage": age,
        "csex": gender_par,
        "cheightmeter": height_cm,
        "ckg": weight,
        "cactivity": td,
        "cgoal":cg,
        "printit": 0,
        "x": 121,
        "y": 28
    }

        

    response = requests.get(url, params=params)

  # Create a Beautiful Soup object
    soup = BeautifulSoup(response.content, 'html.parser')

  # Find the recommended daily calorie intake
    macros = dict()

    labels = soup.find_all('td', {'class': 'arrow_box'})
    values = soup.find_all('td', {'class': 'result_box'})
    for label, value in zip(labels, values):
        temp_lab = label.div.text.strip()
        temp_val = value.text.replace('<', '').replace(':', '').strip().split(" ")
        macros[temp_lab+" ("+temp_val[1]+")"] = temp_val[0]
    # print(macros)
    macros_data.append(macros)


2111it [14:04,  2.50it/s]


In [4]:
macros_df = pd.DataFrame(macros_data)
macros_df

,Protein (grams/day),Carbs (grams/day),Fat (grams/day),Sugar (grams/day),Saturated Fat (grams/day),Food Energy (Calories/dayor)
0,101,222,47,44,19,"1,664"
1,91,199,42,40,17,"1,493"
2,159,349,74,70,30,"2,615"
3,151,331,71,66,28,"2,482"
4,124,272,58,54,23,"2,037"
...,...,...,...,...,...,...
2106,155,339,72,68,29,"2,541"
2107,158,345,74,69,29,"2,591"
2108,158,345,74,69,29,"2,590"
2109,156,342,73,68,29,"2,565"


In [5]:
final_df = df.join(macros_df)
final_df['BMR']=bmr
final_df['TDEE']=tdee
final_df['BMI']=bmi
final_df = final_df.dropna()
final_df

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,NObeyesdad,Protein (grams/day),Carbs (grams/day),Fat (grams/day),Sugar (grams/day),Saturated Fat (grams/day),Food Energy (Calories/dayor),BMR,TDEE,BMI
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,...,Normal_Weight,101,222,47,44,19,"1,664",1450.347000,1740.416400,24.386526
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,...,Normal_Weight,91,199,42,40,17,"1,493",1345.391000,1614.469200,24.238227
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,...,Normal_Weight,159,349,74,70,30,"2,615",1853.180000,2714.908700,23.765432
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,...,Overweight_Level_I,151,331,71,66,28,"2,482",1964.442000,2877.907530,26.851852
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,...,Overweight_Level_II,124,272,58,54,23,"2,037",2020.740600,2424.888720,28.342381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,...,Obesity_Type_III,155,339,72,68,29,"2,541",2101.882087,2522.258504,44.901475
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,...,Obesity_Type_III,158,345,74,69,29,"2,591",2130.839178,2557.007014,43.741923
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,...,Obesity_Type_III,158,345,74,69,29,"2,590",2129.122781,2554.947337,43.543817
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,...,Obesity_Type_III,156,342,73,68,29,"2,565",2114.043816,2536.852580,44.071535


In [6]:
final_df.to_csv("Obesity_with_Macros.csv")
